In [2]:
import pandas as pd
import numpy as np

In [14]:
#read each of the .csv tables
fl_equ = pd.read_csv('FL_opioid_equ.csv')
ga_equ = pd.read_csv('GA_opioid_equ.csv')
counties = pd.read_csv('Counties_DF_Merge_Ready_v2.csv')
deaths = pd.read_csv('overdose_deaths.csv')
controls = pd.read_csv('State_opioid_Controls_Year_Changed.csv')

In [15]:
deaths.describe()

,index,Year,Deaths
count,7920.000000,7920.000000,7920.000000
mean,314.659596,2009.689268,48.578788
std,192.627249,3.656039,74.979139
min,0.000000,2003.000000,10.000000
25%,152.000000,2007.000000,14.000000
50%,304.000000,2010.000000,22.000000
75%,461.000000,2013.000000,48.000000
max,793.000000,2015.000000,862.000000


In [16]:
#merge the counties and population data with the control states table
outer_counties = controls.merge(counties, how = 'inner', on = 'State', indicator = True )

In [21]:
outer_counties.head()
#outer_counties.sample(50)
#outer_counties.describe()
#outer_counties._merge.value_counts()

,State_Full,Year_Changed,State,Unnamed: 0,County,FIPS,Year,Population,_merge
0,California,Control,CA,183,ALAMEDA,6001,2006,1444484.0,both
1,California,Control,CA,184,ALPINE,6003,2006,1276.0,both
2,California,Control,CA,185,AMADOR,6005,2006,38353.0,both
3,California,Control,CA,186,BUTTE,6007,2006,216824.0,both
4,California,Control,CA,187,CALAVERAS,6009,2006,46045.0,both


In [18]:
#Clean the death data table to replace County, Borough, Parish, and city with '' and make them upper case
deaths['County'] = deaths['County'].replace(regex = True, to_replace = [' County', ' Borough', ' Parish', ' City'], value = '')
deaths['County'] = deaths['County'].str.upper()

In [19]:
deaths.sample(10)

,index,State,County,Year,Deaths
1996,124,GA,CHEROKEE,2007.0,22.0
5168,181,IL,MCHENRY,2012.0,40.0
682,273,NM,SANTA FE,2004.0,19.0
4216,576,TX,LUBBOCK,2010.0,19.0
3646,6,AL,JEFFERSON,2010.0,70.0
3969,329,MS,RANKIN,2010.0,25.0
1454,111,GA,CATOOSA,2006.0,10.0
4779,486,OK,LE FLORE,2011.0,14.0
1071,209,ME,PENOBSCOT,2005.0,16.0
6448,73,CA,SONOMA,2014.0,73.0


In [29]:
#drop indicator column
outer_counties = outer_counties.drop(axis = 1, columns = '_merge')

In [30]:
death_added = deaths.merge(outer_counties, how = 'outer', on = ['State', 'Year', 'County'], indicator = True)
death_added._merge.value_counts()
death_added.sample(10)

,index,State,County,Year,Deaths,State_Full,Year_Changed,Unnamed: 0,FIPS,Population,_merge
7009,634.0,TX,BEXAR,2014.0,181.0,NaN,NaN,NaN,NaN,NaN,left_only
11095,NaN,AL,FRANKLIN,2012.0,NaN,Alabama,Control,18887.0,1059.0,31664.0,right_only
4973,680.0,WV,FAYETTE,2011.0,18.0,NaN,NaN,NaN,NaN,NaN,left_only
4897,604.0,TX,KERR,2011.0,10.0,Texas,2007,18366.0,48265.0,49216.0,both
12304,NaN,ID,BLAINE,2011.0,NaN,Idaho,Control,16268.0,16013.0,21354.0,right_only
10605,NaN,GA,WALTON,2011.0,NaN,Georgia,Control,16245.0,13297.0,83003.0,right_only
12032,NaN,OR,TILLAMOOK,2010.0,NaN,Oregon,Control,14804.0,41057.0,25200.0,right_only
12932,NaN,MO,DADE,2011.0,NaN,Montana,Control,17222.0,29057.0,7896.0,right_only
5142,155.0,GA,MUSCOGEE,2012.0,21.0,Georgia,Control,19347.0,13215.0,191278.0,both
2378,506.0,TX,SMITH,2007.0,18.0,Texas,2007,5873.0,48423.0,199953.0,both


In [34]:
death_added.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13280 entries, 0 to 13279
Data columns (total 10 columns):
index           7920 non-null float64
State           13280 non-null object
County          13280 non-null object
Year            13280 non-null float64
Deaths          7920 non-null float64
State_Full      6783 non-null object
Year_Changed    6783 non-null object
Unnamed: 0      6783 non-null float64
FIPS            6783 non-null float64
Population      6783 non-null float64
dtypes: float64(6), object(4)
memory usage: 1.1+ MB


In [35]:
#rename columns to match with the intermediate merged table for Florida
fl_equ = fl_equ.rename(columns = {'BUYER_COUNTY':'County', 'County_FIPS':'FIPS'})
fl_equ.head()

,Year,FIPS,County,Morphine_equ,State
0,2006,12001,ALACHUA,82596.618688,FL
1,2006,12003,BAKER,11900.178638,FL
2,2006,12005,BAY,61998.102472,FL
3,2006,12007,BRADFORD,13424.830895,FL
4,2006,12009,BREVARD,201186.153403,FL


In [36]:
#rename columns to match with the intermediate merged table for Georgia
ga_equ = ga_equ.rename(columns = {'BUYER_COUNTY':'County', 'County_FIPS':'FIPS'})
ga_equ.head()

,Year,FIPS,County,Morphine_equ,State
0,2006,13001,APPLING,3996.843779,GA
1,2006,13003,ATKINSON,1375.483686,GA
2,2006,13005,BACON,4586.778211,GA
3,2006,13007,BAKER,9.651600,GA
4,2006,13009,BALDWIN,11069.705010,GA


In [46]:
#drop indicator column from death_added
death_added = death_added.drop(axis = 1, columns = '_merge')

KeyError: "['_merge'] not found in axis"

In [53]:
#merge florida morphine and georgia morphine
m_combined = ga_equ.merge(fl_equ, how = 'outer', on = ['County', 'State', 'Morphine_equ','FIPS', 'Year'], indicator = True)
m_combined.sample(10)

,Year,FIPS,County,Morphine_equ,State,_merge
703,2010,13199,MERIWETHER,5493.199980,GA,left_only
830,2011,13151,HENRY,81436.830702,GA,left_only
784,2011,13055,CHATTOOGA,14430.830404,GA,left_only
739,2010,13279,TOOMBS,17936.853413,GA,left_only
88,2006,13183,LONG,486.602287,GA,left_only
281,2007,13271,TELFAIR,5755.375163,GA,left_only
1077,2006,12031,DUVAL,293452.975473,FL,right_only
26,2006,13057,CHEROKEE,39825.913978,GA,left_only
426,2008,13253,SEMINOLE,2698.703822,GA,left_only
488,2009,13067,COBB,205475.851632,GA,left_only


In [52]:
#check merge
m_combined._merge.value_counts()

In [55]:
#drop _merge column
m_combined = m_combined.drop(axis = 1, columns = '_merge')

KeyError: "['_merge'] not found in axis"

In [58]:
#add the morphine table into the intermediate table
morphine_added = m_combined.merge(death_added, how = 'outer', on = ['County', 'State', 'FIPS','Year'], indicator = True)
morphine_added.describe()

,Year,FIPS,Morphine_equ,index,Deaths,Unnamed: 0,Population
count,13294.000000,6797.000000,1.529000e+03,7920.000000,7920.000000,6783.000000,6.783000e+03
mean,2009.396194,27524.487127,7.207393e+04,314.659596,48.578788,10832.489164,1.263234e+05
std,3.116705,16989.340559,2.272125e+05,192.627249,74.979139,6368.061747,4.524903e+05
min,2003.000000,1001.000000,3.027000e-01,0.000000,10.000000,0.000000,4.100000e+01
25%,2007.000000,13073.000000,3.931383e+03,152.000000,14.000000,5712.500000,1.186750e+04
50%,2009.000000,28147.000000,1.178754e+04,304.000000,22.000000,10900.000000,2.538400e+04
75%,2012.000000,48101.000000,4.509421e+04,461.000000,48.000000,16133.500000,7.406650e+04
max,2015.000000,53077.000000,3.026737e+06,793.000000,862.000000,21847.000000,9.840024e+06


In [59]:
morphine_added.head()

,Year,FIPS,County,Morphine_equ,State,index,Deaths,State_Full,Year_Changed,Unnamed: 0,Population,_merge
0,2006,13001.0,APPLING,3996.843779,GA,NaN,NaN,Georgia,Control,384.0,17725.0,both
1,2006,13003.0,ATKINSON,1375.483686,GA,NaN,NaN,Georgia,Control,385.0,8197.0,both
2,2006,13005.0,BACON,4586.778211,GA,NaN,NaN,Georgia,Control,386.0,10682.0,both
3,2006,13007.0,BAKER,9.651600,GA,NaN,NaN,Georgia,Control,387.0,3877.0,both
4,2006,13009.0,BALDWIN,11069.705010,GA,NaN,NaN,Georgia,Control,388.0,46513.0,both


In [60]:
morphine_added._merge.value_counts()

right_only    11765
both           1515
left_only        14
Name: _merge, dtype: int64

In [61]:
#remove merge column
morphine_added = morphine_added.drop(axis = 1, columns = '_merge')

In [62]:
#Drop the unnamed column, which was an index column left over from the import of an intermediate table
final_frame = morphine_added.drop(columns = 'Unnamed: 0')
final_frame.sample(10)

,Year,FIPS,County,Morphine_equ,State,index,Deaths,State_Full,Year_Changed,Population
1717,2003,NaN,WASHTENAW,NaN,MI,188.0,15.0,NaN,NaN,NaN
4763,2009,NaN,FAIRFIELD,NaN,OH,406.0,11.0,NaN,NaN,NaN
1582,2003,NaN,PUEBLO,NaN,CO,53.0,10.0,NaN,NaN,NaN
9459,2006,48131.0,DUVAL,NaN,TX,NaN,NaN,Texas,2007,12298.0
8015,2014,NaN,WAYNE,NaN,OH,510.0,13.0,NaN,NaN,NaN
12312,2010,16085.0,VALLEY,NaN,ID,NaN,NaN,Idaho,Control,9846.0
4844,2009,NaN,HORRY,NaN,SC,487.0,37.0,NaN,NaN,NaN
12685,2008,29167.0,POLK,NaN,MO,NaN,NaN,Montana,Control,30790.0
9695,2007,48163.0,FRIO,NaN,TX,NaN,NaN,Texas,2007,16776.0
5187,2010,NaN,KALAMAZOO,NaN,MI,288.0,21.0,NaN,NaN,NaN


In [63]:
#add in a deaths per capita column
final_frame['Deaths_PC'] = final_frame['Deaths'] / final_frame['Population']
final_frame.sample(10)

,Year,FIPS,County,Morphine_equ,State,index,Deaths,State_Full,Year_Changed,Population,Deaths_PC
8116,2014,NaN,DEKALB,NaN,TN,611.0,10.0,NaN,NaN,NaN,NaN
8460,2015,NaN,DELAWARE,NaN,IN,204.0,32.0,NaN,NaN,NaN,NaN
2582,2005,NaN,HAMPDEN,NaN,MA,191.0,39.0,NaN,NaN,NaN,NaN
3180,2006,NaN,JEFFERSON,NaN,OH,357.0,11.0,NaN,NaN,NaN,NaN
5194,2010,NaN,OAKLAND,NaN,MI,295.0,192.0,NaN,NaN,NaN,NaN
8330,2015,NaN,VENTURA,NaN,CA,74.0,119.0,NaN,NaN,NaN,NaN
11621,2009,28055.0,ISSAQUENA,NaN,MS,NaN,NaN,Mississippi,Control,2130.0,NaN
4536,2009,NaN,ALLEN,NaN,IN,179.0,31.0,NaN,NaN,NaN,NaN
8283,2015,NaN,COCONINO,NaN,AZ,27.0,15.0,NaN,NaN,NaN,NaN
879,2011,13253.0,SEMINOLE,3653.139897,GA,NaN,NaN,Georgia,Control,8784.0,NaN


In [64]:
final_frame.describe()

,Year,FIPS,Morphine_equ,index,Deaths,Population,Deaths_PC
count,13294.000000,6797.000000,1.529000e+03,7920.000000,7920.000000,6.783000e+03,1423.000000
mean,2009.396194,27524.487127,7.207393e+04,314.659596,48.578788,1.263234e+05,0.000145
std,3.116705,16989.340559,2.272125e+05,192.627249,74.979139,4.524903e+05,0.000080
min,2003.000000,1001.000000,3.027000e-01,0.000000,10.000000,4.100000e+01,0.000014
25%,2007.000000,13073.000000,3.931383e+03,152.000000,14.000000,1.186750e+04,0.000093
50%,2009.000000,28147.000000,1.178754e+04,304.000000,22.000000,2.538400e+04,0.000127
75%,2012.000000,48101.000000,4.509421e+04,461.000000,48.000000,7.406650e+04,0.000175
max,2015.000000,53077.000000,3.026737e+06,793.000000,862.000000,9.840024e+06,0.000746


In [228]:
#export the final frame to a csv
#final_frame.to_csv('Final_Data_Frame_Python_Midterm.csv')